In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz 
import os
import json
from pathlib import Path
import spacy
import en_core_web_sm
import nltk
import re
nlp = en_core_web_sm.load()
nlp.max_length = 6000000 

In [2]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [6]:
query = "SELECT * FROM pcmr.issues where tableId IN ('02db9f91-572a-44af-9858-4add101353c1','03bfc26a-c6d0-4761-b8f5-47acf2290d02', '082134c0-6a4b-425b-a4ae-e79acd7316cb', '0d10a967-88d6-42e5-9bd7-309f24022b5f', '333e1e53-8897-41fa-acbd-86be8afb31c7', '35bd2caf-562c-4d14-a5d6-373f168b4acb', '397db969-9996-4d9e-bb05-6df69d0fe4a4', '417546c4-dacf-4c12-ae75-4dc4e656e198', '491c36c1-82d4-46ae-a684-470915a5659b', '60b3993d-7075-4790-8519-ba8193579754', '64a7ba33-ceee-4593-87a3-8f08dd46c8f4', '67691780-af41-414b-a0c2-aa33a3442cdc', '6a2f1370-1cd5-4ebb-a4bd-a1fe9d5a516a', '6b437f67-967b-4ef6-bd28-5ac8d39138e4', '77cc0b8d-8244-4622-8d9d-a56daf6069e8', '8bb683d9-f7ee-4a54-ad3d-dddc61ccdfcf', '9476acc2-294a-4cd6-a952-8274aedb645a', 'a6623233-9c9f-436b-ad11-0987ab3825e7', 'c04807de-2df1-4d26-9352-70d3cb6cb10b', 'cb197d7e-3ef6-4ee0-93d1-504c7286b580', 'f143c6b8-cf77-41c1-88b2-e7c97ba657c1', 'f2ebd484-4ec2-4481-907d-17334ca4657f', 'f4db9fc5-3a73-499a-ab1e-ab643530ea99', 'fdb3d057-943a-4fab-99ac-1f4eed471512', '44a33e5f-d99e-48ef-ad56-bbb516ec8796', 'bfafbfd0-8bb5-4283-8f5e-dd7cbcec480c');"
with engine.connect() as conn:
    df = pd.read_sql(query, conn)[:1]
df

,tableId,rowIndex,content,issue_pri,issue_sec,subvec_standardized,land_use,land_use_standardized,loc_coord,loc_kp,...,loc_utm,status_bin,status_txt,vec_pri,vec_sec,status,vec_simple,subvec_simple,imputation,pipelineName
0,02db9f91-572a-44af-9858-4add101353c1,1,"[[""Legal Location"", ""1<s>st</s> Year Issue/ Co...",Industrial/Noxious Weeds (KP 0+000 to 0+100),None,None,None,None,NW-25 -91-12 W4,Industrial/Noxious Weeds (KP 0+000 to 0+100),...,None,None,"June 22, 2016 – Intermittent weeds observed, r...",Industrial/Noxious Weeds (KP 0+000 to 0+100),None,Resolved,vegetation,weeds,0,MOOSA CROSSOVER


In [ ]:
def clean_dataframe(dataframe, column_name):
    dataframe[column_name] = dataframe[column_name].str.lower()
    pattern = '|'.join(['<s>', '</s>',])
    dataframe[column_name] = dataframe[column_name].str.replace(pattern, '')
    dataframe[column_name] = dataframe[column_name].str.replace('\(s\)', 's') #reference: https://stackoverflow.com/questions/51440233/how-to-remove-the-values-which-are-in-parentheses-in-pandashttps://stackoverflow.com/questions/51440233/how-to-remove-the-values-which-are-in-parentheses-in-pandas
    return dataframe    

In [ ]:
query = "SELECT * FROM issues WHERE tableId = '3e1c53b4-5c01-46e2-bd72-5a338b5852f9';"

with engine.connect() as conn:
    df = pd.read_sql(query, conn)
    #clean_dataframe(df, 'table_content')
dic = {}

for row in df.itertuples():
    # converting JSON string to a list of lists of strings
    table = json.loads(row.table_content)
    headers = table[0]  # column headers  
    for header in headers:
        #header = lemmawordnet(header)
        #header = clean_header(header)
        #header = lemmaspacy(header)
        #header = re.sub(r'\(.*?\)', lambda x: ''.join(x.group(0).split()), header) # removing whitespace between parentheses (reference: https://stackoverflow.com/questions/34088489/how-to-remove-whitespace-inside-brackethttps://stackoverflow.com/questions/34088489/how-to-remove-whitespace-inside-bracket)
        header = " ".join(header.split())
        if header in dic:
            dic[header] += 1
        else:
            dic[header] = 1

my_list = [(header, count) for header, count in dic.items()]  # Converting to list
my_list.sort(key=lambda tup: tup[1], reverse=True)  # sorting the list

total_headers = 0
print(f"COUNT\tHEADER")
for item in my_list:
    total_headers += item[1]
    print(f"{item[1]}\t{item[0]}")
print()
print(f'Total headers: {total_headers}; unique headers: {len(my_list)}')

In [ ]:
df3 = pd.DataFrame(my_list)
df3.to_csv('headers4.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
for row in df.itertuples():
    table = json.loads(row.table_content)
    print(type(table))
    lst_element1 = "SPREAD"
    lst_element2 = "QUARTER SECTION"
    if lst_element1 in table[0][0] and lst_element2 in table[0][1]:
        print(row.tableId)

In [ ]:
for row in df.itertuples():
    table = json.loads(row.table_content)
    if table[0][len(table[0])-1] == "Topic":
        data = pd.DataFrame(table)
        data.to_csv(r"C:\Users\t1nipun\Desktop\PCMR\human-robot\Data_Analysis\csvs\\" + row.tableId + '.csv', encoding = 'utf-8-sig', index = False, header = None)
    else:
        table[0].extend(['VEC', 'GIS', 'Topic'])
        for i in range(1, len(table)-1):
            table[i].extend(['','',''])
        data = pd.DataFrame(table)
        data.to_csv(r"C:\Users\t1nipun\Desktop\PCMR\human-robot\Data_Analysis\csvs\\" + row.tableId + '.csv', encoding = 'utf-8-sig', index = False, header = None)

In [ ]:
!pip install opencv-python
!pip install camelot-py[all] 
# Please install ghostcript using this link: https://www.ghostscript.com/
import camelot
import tkinter
#tables = camelot.read_pdf('G:/Post Construction/george.pdf', pages = '7', line_scale=40, flag_size=True, copy_text=['v'],) # latice method
tables = camelot.read_pdf('G:/Post Construction/george.pdf', flavor = 'stream', edge_tol=500, pages = '6-38') # stream method
tables.export('scheduleA.csv', f='csv', compress=True)

In [ ]:
tables.export('scheduleA.csv', f='csv', compress=True) # json, excel, html
tables[0]

In [ ]:
tables[0].parsing_report

In [ ]:
tables[0].to_csv('george1.csv')
tables[0].df